In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key



In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/citieschallenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Palimbang,PH,6.21,124.19,79.74,78,97,4.09,light rain,0.22,0
1,1,Vao,NC,-22.67,167.48,69.80,76,73,19.15,broken clouds,0.00,0
2,2,Tupancireta,BR,-29.08,-53.84,55.38,97,100,9.22,light rain,0.29,0
3,3,Huarmey,PE,-10.07,-78.15,63.07,76,1,9.04,clear sky,0.00,0
4,4,Mahebourg,MU,-20.41,57.70,69.80,73,40,14.99,scattered clouds,0.00,0


In [3]:
# Get the data types
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [4]:
city_data_df['Rain (inches)'] = city_data_df['Rain (inches)'].astype(object)


In [5]:
city_data_df['Snow (inches)'] = city_data_df['Snow (inches)'].astype(object)


In [6]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)           object
Snow (inches)           object
dtype: object

In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Ask the customer to add a minimum and maximum temperature value, rain and snow 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
Rain = str(input("Do you want it to be raining?(yes/no)"))
Snow = str(input("Do you want it to be snowing?(yes/no)"))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining?(yes/no)no
Do you want it to be snowing?(yes/no)no


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain (inches)"] == 0 if Rain == "no" else city_data_df["Rain (inches)"] > 0) &
                                       (city_data_df["Snow (inches)"] == 0 if Snow == "no" else city_data_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
8,8,Kahului,US,20.89,-156.47,89.60,43,40,24.16,scattered clouds,0,0
11,11,Pochutla,MX,15.74,-96.47,87.01,66,40,11.41,scattered clouds,0,0
12,12,Yellowknife,CA,62.46,-114.35,75.00,46,75,13.87,broken clouds,0,0
14,14,Makakilo City,US,21.35,-158.09,87.80,52,75,5.82,broken clouds,0,0
18,18,Georgetown,MY,5.41,100.34,77.00,88,20,4.52,few clouds,0,0
27,27,Guerrero Negro,MX,27.98,-114.06,87.62,31,0,17.49,clear sky,0,0
28,28,Kapaa,US,22.08,-159.32,82.99,78,90,14.99,overcast clouds,0,0
31,31,Samana,DO,19.21,-69.34,82.40,83,40,6.93,scattered clouds,0,0
32,32,Buala,SB,-8.14,159.59,83.17,75,81,7.65,broken clouds,0,0
35,35,Ixtapa,MX,20.70,-105.20,89.60,66,90,10.29,overcast clouds,0,0


In [10]:
preferred_cities_df.count()

City_ID                165
City                   165
Country                165
Lat                    165
Lng                    165
Max Temp               165
Humidity               165
Cloudiness             165
Wind Speed             165
Current Description    165
Rain (inches)          165
Snow (inches)          165
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
8,Kahului,US,89.60,20.89,-156.47,scattered clouds,
11,Pochutla,MX,87.01,15.74,-96.47,scattered clouds,
12,Yellowknife,CA,75.00,62.46,-114.35,broken clouds,
14,Makakilo City,US,87.80,21.35,-158.09,broken clouds,
18,Georgetown,MY,77.00,5.41,100.34,few clouds,
27,Guerrero Negro,MX,87.62,27.98,-114.06,clear sky,
28,Kapaa,US,82.99,22.08,-159.32,overcast clouds,
31,Samana,DO,82.40,19.21,-69.34,scattered clouds,
32,Buala,SB,83.17,-8.14,159.59,broken clouds,
35,Ixtapa,MX,89.60,20.70,-105.20,overcast clouds,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found... skipping.")

hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.


In [14]:
hotel_df.head(20)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
8,Kahului,US,89.60,20.89,-156.47,scattered clouds,Maui Seaside Hotel
11,Pochutla,MX,87.01,15.74,-96.47,scattered clouds,Hotel Posada San Jose
12,Yellowknife,CA,75.00,62.46,-114.35,broken clouds,The Explorer Hotel
14,Makakilo City,US,87.80,21.35,-158.09,broken clouds,Marriott's Ko Olina Beach Club
18,Georgetown,MY,77.00,5.41,100.34,few clouds,Cititel Penang
27,Guerrero Negro,MX,87.62,27.98,-114.06,clear sky,"Casa Laguna, Bed & Breakfast"
28,Kapaa,US,82.99,22.08,-159.32,overcast clouds,Sheraton Kauai Resort at Coconut Beach
31,Samana,DO,82.40,19.21,-69.34,scattered clouds,The Bannister - Hotel & Yacht Club
32,Buala,SB,83.17,-8.14,159.59,broken clouds,Maringe Lagoon Lodge
35,Ixtapa,MX,89.60,20.70,-105.20,overcast clouds,Container Inn Puerto Vallarta


In [15]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd> & {Max Temp} °F</dd>
<dt>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPY_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")